### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

### Load the dataset

In [2]:
sales = pd.read_csv('sales.csv')
sales.head()

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Load the dictionary created from API and convert to dataframe

In [3]:
games_dict_df = pd.DataFrame.from_dict(
    np.load('games_dict.npy', allow_pickle=True).item(), orient='index'
).reset_index(level=0).rename(columns={'index':'name'})
games_dict_df.head()

,name,release_date,metacritic_score
0,Wii Sports,2006-11-19,NaN
1,Super Mario Bros.,1985-09-13,NaN
2,Mario Kart Wii,2008-04-27,NaN
3,Wii Sports Resort,2009-07-26,NaN
4,Pokemon Red/Pokemon Blue,1996-02-27,NaN


### Join `sales` and `games_dict_df`

In [4]:
game_sales = pd.merge(sales, games_dict_df, how='left', left_on='name', right_on='name')
game_sales.head()

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,release_date,metacritic_score
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2006-11-19,NaN
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,1985-09-13,NaN
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,2008-04-27,NaN
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,2009-07-26,NaN
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,1996-02-27,NaN


### Explore the dataset and the target website

I noticed that a lot of games are missing their `metacritic_score` (they weren't uploaded to the database from where I pulled the information via API calls), so I'm going directly to the source, [Metacritic](http://metacritic.com). Similarly to the API exercise, I'm filtering my dataset to those games where `global_sales > 5` to limit the number of attempts.

In [5]:
tmp_games = game_sales[game_sales['global_sales']>5]
missing_metacritic_score = tmp_games[pd.isnull(tmp_games['metacritic_score'])].reset_index(drop=True)
missing_metacritic_score[['name', 'platform', 'global_sales', 'metacritic_score']].head()

,name,platform,global_sales,metacritic_score
0,Wii Sports,Wii,82.74,NaN
1,Super Mario Bros.,NES,40.24,NaN
2,Mario Kart Wii,Wii,35.82,NaN
3,Wii Sports Resort,Wii,33.00,NaN
4,Pokemon Red/Pokemon Blue,GB,31.37,NaN


The links on Metacritic are constructed in a way that requires the platform as well as the name of the game without special characters and with hyphens instead of spaces. I'm adding a column to the dataset with the constructed URL.

* Create a function to take care of mappings and URL construction with the help of the `platform_dict` mapping dictionary

In [10]:
platform_dict = {
    'GBA': 'game-boy-advance', 
    'XB': 'xbox',
    'X360': 'xbox-360',
    'PS': 'playstation',
    'PS2': 'playstation-2',
    'PS3': 'playstation-3',
    'PS4': 'playstation-4',
    'N64': 'nintendo-64',
    'GC': 'gamecube',
    'WiiU': 'wii-u'
}

In [22]:
def construct_metacritic_url(platform, name):       
    # Names don't have special characters and use hyphens instead of spaces
    url_name = name.replace(' ', '-').replace('/', '-').replace('.', '')
    
    # Platforms are not abbreviated, using the platform_dict mapping directory instead
    try:
        url_platform = platform_dict[platform]
    except KeyError:
        url_platform = platform
        
    return('http://metacritic.com/game/'+url_platform.lower()+'/'+url_name.lower())

In [23]:
missing_metacritic_score['metacritic_url'] = missing_metacritic_score.apply(
    lambda x: construct_metacritic_url(x['platform'], x['name']), axis=1
)
missing_metacritic_score[['name', 'platform', 'metacritic_url']].head()

,name,platform,metacritic_url
0,Wii Sports,Wii,http://metacritic.com/game/wii/wii-sports
1,Super Mario Bros.,NES,http://metacritic.com/game/nes/super-mario-bros
2,Mario Kart Wii,Wii,http://metacritic.com/game/wii/mario-kart-wii
3,Wii Sports Resort,Wii,http://metacritic.com/game/wii/wii-sports-resort
4,Pokemon Red/Pokemon Blue,GB,http://metacritic.com/game/gb/pokemon-red-poke...


* Explore the website with a specific example

In [ ]:
res = requests.get('http://metacritic.com/game/wii/wii-sports', headers={'User-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(res.text, 'html.parser')
print(soup.prettify())

The metascore on Metacritic is in a div class called `metascore_w`.

In [ ]:
soup.find_all('div', class_='metascore_w')

But that returns all scores from the reviews as well, however, it is the only score that uses the `xlarge` class, so I can extract that.

In [ ]:
soup.find_all('div', class_='xlarge')[0].text

### Create a loop to pull all available scores from Metacritic to a dictionary

* Utilizing `time.sleep` with a random number between 1 and 1.5 seconds to avoid ban for scraping
* The `meta_key` that I use for the dictionary's key has to be a combination of the `name` and the `platform` from the original dataset, because one game can show up on the list for multiple platforms
* Not all games are available on Metacritic (especially older ones), so I need to do some error-handling for 404 response
* There are games that haven't been reviewed yet, so I have to see if `xlarge` returns anything first, before pulling the text

In [ ]:
header = {'User-agent': 'Mozilla/5.0'}
metacritic_dict = {}

for i in range(len(missing_metacritic_score)):
    meta_key = missing_metacritic_score['name'][i]+'_'+missing_metacritic_score['platform'][i]
    
    time.sleep(np.random.uniform(1, 1.5))
    r = requests.get(missing_metacritic_score['metacritic_url'][i], headers=header)
    
    if r.status_code == 404:
        continue
    else:  
        xlarge_div = BeautifulSoup(r.text, 'html.parser').find_all('div', class_='xlarge')
        if len(xlarge_div) > 0:
            meta_score = xlarge_div[0].text  
        else:
            meta_score = None
        
        metacritic_dict.update({meta_key: meta_score})

In [ ]:
metacritic_dict

### Save the dictionary

In [ ]:
np.save('metacritic_dict.npy', metacritic_dict)